# Notebook 02 - Preprocessing & Feature Engineering

**Objective:** Clean, filter, and create **4 processed datasets** with different data source combinations.

---

**4 Datasets Produced:**

| Dataset | Sources | Key |
|---|---|---|
| `movies_processed_metadata.csv` | movies_metadata only | `metadata` |
| `movies_processed_meta_credits.csv` | movies_metadata + credits | `meta_credits` |
| `movies_processed_meta_keywords.csv` | movies_metadata + keywords | `meta_keywords` |
| `movies_processed.csv` | all three combined | `all` |

**Preprocessing:**
1. Load & parse JSON columns in all 3 raw datasets
2. Type conversions, drop corrupted rows (non-numeric IDs)
3. Remove duplicates from all 3 datasets
4. Filter: Released status, budget > 0, revenue > 0, vote_count > 0

**Feature Engineering (applied to base metadata before merging):**
5. Binary: `is_collection`, `is_english`
6. Temporal: `release_year`, `release_month`
7. Financial: `roi` (replaces raw revenue)
8. Counts: `num_genres`, `num_production_companies`, etc.
9. `primary_genre` categorical

**Additional Features (for datasets with credits):**
10. `num_cast`, `num_crew`
11. Top modern directors & actors binary features

**Additional Features (for datasets with keywords):**
12. `num_keywords`

In [2]:
# 1. Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from ast import literal_eval
import os
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print('Libraries loaded')

Libraries loaded


In [3]:
# Load all three raw datasets
movies_df = pd.read_csv('../data/raw/movies_metadata.csv', low_memory=False)
credits_df = pd.read_csv('../data/raw/credits.csv')
keywords_df = pd.read_csv('../data/raw/keywords.csv')

print(f'movies_metadata : {movies_df.shape[0]:,} rows x {movies_df.shape[1]} cols')
print(f'credits         : {credits_df.shape[0]:,} rows x {credits_df.shape[1]} cols')
print(f'keywords        : {keywords_df.shape[0]:,} rows x {keywords_df.shape[1]} cols')

movies_metadata : 45,466 rows x 24 cols
credits         : 45,476 rows x 3 cols
keywords        : 46,419 rows x 2 cols


In [4]:
# JSON parser for stringified list-of-dicts columns
def parse_json_column(val):
    if pd.isna(val):
        return []
    if isinstance(val, str):
        try:
            return json.loads(val)
        except Exception:
            try:
                parsed = literal_eval(val)
                return parsed if isinstance(parsed, list) else []
            except Exception:
                return []
    return val if isinstance(val, list) else []

# Parse movies JSON columns
print('Parsing JSON columns...')
for col in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    movies_df[col] = movies_df[col].apply(parse_json_column)

# Parse credits JSON columns
credits_df['cast_parsed'] = credits_df['cast'].apply(parse_json_column)
credits_df['crew_parsed'] = credits_df['crew'].apply(parse_json_column)

# Parse keywords
keywords_df['keywords_parsed'] = keywords_df['keywords'].apply(parse_json_column)

print('Done')

Parsing JSON columns...
Done


---
## 2. Cleaning & Filtering

In [5]:
# 2.1  Type conversions + drop corrupted rows
n_start = len(movies_df)
print(f'Starting rows: {n_start:,}')

# Drop rows with non-numeric IDs (3 corrupted rows found in EDA)
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
movies_df = movies_df.dropna(subset=['id'])
movies_df['id'] = movies_df['id'].astype(int)

# Convert numeric columns stored as strings
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')
movies_df['popularity'] = pd.to_numeric(movies_df['popularity'], errors='coerce')
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
movies_df['runtime'] = pd.to_numeric(movies_df['runtime'], errors='coerce')
movies_df['vote_average'] = pd.to_numeric(movies_df['vote_average'], errors='coerce')
movies_df['vote_count'] = pd.to_numeric(movies_df['vote_count'], errors='coerce')

# Parse release date
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

n_after = len(movies_df)
print(f'After type cleanup: {n_after:,}  (dropped {n_start - n_after:,} corrupted rows)')

Starting rows: 45,466
After type cleanup: 45,463  (dropped 3 corrupted rows)


In [6]:
# 2.2  Remove duplicates from all 3 datasets
print('--- Removing duplicates ---')

n1 = len(movies_df)
movies_df = movies_df.drop_duplicates(subset=['id'], keep='first')
print(f'Movies:   {n1:,} -> {len(movies_df):,}  (dropped {n1 - len(movies_df):,})')

n2 = len(credits_df)
credits_df = credits_df.drop_duplicates(subset=['id'], keep='first')
print(f'Credits:  {n2:,} -> {len(credits_df):,}  (dropped {n2 - len(credits_df):,})')

n3 = len(keywords_df)
keywords_df = keywords_df.drop_duplicates(subset=['id'], keep='first')
print(f'Keywords: {n3:,} -> {len(keywords_df):,}  (dropped {n3 - len(keywords_df):,})')

--- Removing duplicates ---
Movies:   45,463 -> 45,433  (dropped 30)
Credits:  45,476 -> 45,432  (dropped 44)
Keywords: 46,419 -> 45,432  (dropped 987)


In [7]:
# 2.3  Filter movies
print('--- Filtering movies ---')
print(f'Start: {len(movies_df):,}')

# Keep only Released movies
movies_df = movies_df[movies_df['status'] == 'Released']
print(f'After status=Released: {len(movies_df):,}')

# Remove movies with budget <= 0 or revenue <= 0 (corrupted/missing financial data)
movies_df = movies_df[(movies_df['budget'] > 0) & (movies_df['revenue'] > 0)]
print(f'After budget>0 & revenue>0: {len(movies_df):,}')

# Remove entries with 0 vote_count
movies_df = movies_df[movies_df['vote_count'] > 0]
print(f'After vote_count>0: {len(movies_df):,}')

# Handle runtime: drop zeros, fill NaN with median
movies_df = movies_df[movies_df['runtime'] > 0]
med_runtime = movies_df['runtime'].median()
movies_df['runtime'] = movies_df['runtime'].fillna(med_runtime)
print(f'After valid runtime: {len(movies_df):,}')

# Drop missing release dates
movies_df = movies_df.dropna(subset=['release_date'])
print(f'Final after all filters: {len(movies_df):,}')

--- Filtering movies ---
Start: 45,433
After status=Released: 44,985
After budget>0 & revenue>0: 5,371
After vote_count>0: 5,364
After valid runtime: 5,352
Final after all filters: 5,352


---
## 3. Base Feature Engineering (Metadata)

In [8]:
# 3.1  Binary, temporal, financial & count features from metadata

print('--- Binary Features ---')
movies_df['is_collection'] = movies_df['belongs_to_collection'].apply(
    lambda x: 0 if pd.isna(x) or x == '' else 1)
movies_df['is_english'] = (movies_df['original_language'] == 'en').astype(int)
print(f'is_collection: {movies_df["is_collection"].sum():,} / {len(movies_df):,}')
print(f'is_english:    {movies_df["is_english"].sum():,} / {len(movies_df):,}')

print('\n--- Temporal Features ---')
movies_df['release_year'] = movies_df['release_date'].dt.year.astype(int)
movies_df['release_month'] = movies_df['release_date'].dt.month.astype(int)
print(f'Year range: {movies_df["release_year"].min()} – {movies_df["release_year"].max()}')

print('\n--- Financial Feature: ROI ---')
movies_df['roi'] = ((movies_df['revenue'] - movies_df['budget']) / movies_df['budget'] * 100).round(2)
movies_df = movies_df.drop(columns=['revenue'])
print(f'ROI median={movies_df["roi"].median():.1f}%, mean={movies_df["roi"].mean():.1f}%')
print(f'Profitable (ROI>0): {(movies_df["roi"]>0).sum():,} / {len(movies_df):,}')

print('\n--- Count Features (Metadata) ---')
movies_df['num_genres'] = movies_df['genres'].apply(lambda x: len(x) if isinstance(x, list) else 0)
movies_df['num_production_companies'] = movies_df['production_companies'].apply(lambda x: len(x) if isinstance(x, list) else 0)
movies_df['num_production_countries'] = movies_df['production_countries'].apply(lambda x: len(x) if isinstance(x, list) else 0)
movies_df['num_spoken_languages'] = movies_df['spoken_languages'].apply(lambda x: len(x) if isinstance(x, list) else 0)

count_cols = ['num_genres', 'num_production_companies', 'num_production_countries', 'num_spoken_languages']
print(movies_df[count_cols].describe().round(1).to_string())

print('\n--- Primary Genre ---')
def get_primary_genre(genres_list):
    if isinstance(genres_list, list) and len(genres_list) > 0:
        return genres_list[0].get('name', 'Unknown')
    return 'Unknown'

movies_df['primary_genre'] = movies_df['genres'].apply(get_primary_genre)
print(f'Unique genres: {movies_df["primary_genre"].nunique()}')
print(movies_df['primary_genre'].value_counts().head(5).to_string())

--- Binary Features ---
is_collection: 1,220 / 5,352
is_english:    4,783 / 5,352

--- Temporal Features ---
Year range: 1915 – 2017

--- Financial Feature: ROI ---
ROI median=106.7%, mean=559504.6%
Profitable (ROI>0): 3,743 / 5,352

--- Count Features (Metadata) ---
       num_genres  num_production_companies  num_production_countries  num_spoken_languages
count      5352.0                    5352.0                    5352.0                5352.0
mean          2.6                       2.9                       1.4                   1.5
std           1.1                       2.2                       0.8                   0.9
min           0.0                       0.0                       0.0                   0.0
25%           2.0                       1.0                       1.0                   1.0
50%           3.0                       2.0                       1.0                   1.0
75%           3.0                       4.0                       2.0                   

---
## 4. Create 4 Processed Datasets

In [9]:
# 4.1  Prepare credits features
credits_merge = credits_df[['id', 'cast_parsed', 'crew_parsed']].copy()
credits_merge['id'] = pd.to_numeric(credits_merge['id'], errors='coerce')
credits_merge = credits_merge.dropna(subset=['id'])
credits_merge['id'] = credits_merge['id'].astype(int)
credits_merge['num_cast'] = credits_merge['cast_parsed'].apply(lambda x: len(x) if isinstance(x, list) else 0)
credits_merge['num_crew'] = credits_merge['crew_parsed'].apply(lambda x: len(x) if isinstance(x, list) else 0)
print(f'Credits features ready: {len(credits_merge):,} rows')

# 4.2  Prepare keywords features
keywords_merge = keywords_df[['id', 'keywords_parsed']].copy()
keywords_merge['id'] = pd.to_numeric(keywords_merge['id'], errors='coerce')
keywords_merge = keywords_merge.dropna(subset=['id'])
keywords_merge['id'] = keywords_merge['id'].astype(int)
keywords_merge['num_keywords'] = keywords_merge['keywords_parsed'].apply(lambda x: len(x) if isinstance(x, list) else 0)
print(f'Keywords features ready: {len(keywords_merge):,} rows')

# 4.3  Create 4 datasets by merging different combinations
print('\n--- Creating 4 datasets ---')

# Dataset 1: Metadata Only
df_metadata = movies_df.copy()
print(f'1. Metadata only:         {len(df_metadata):,} rows')

# Dataset 2: Metadata + Credits
df_meta_credits = movies_df.merge(credits_merge, on='id', how='inner')
print(f'2. Metadata + Credits:    {len(df_meta_credits):,} rows (lost {len(movies_df) - len(df_meta_credits):,})')

# Dataset 3: Metadata + Keywords
df_meta_keywords = movies_df.merge(keywords_merge, on='id', how='inner')
print(f'3. Metadata + Keywords:   {len(df_meta_keywords):,} rows (lost {len(movies_df) - len(df_meta_keywords):,})')

# Dataset 4: All Combined
df_all = movies_df.merge(credits_merge, on='id', how='inner')
df_all = df_all.merge(keywords_merge, on='id', how='inner')
print(f'4. All Combined:          {len(df_all):,} rows (lost {len(movies_df) - len(df_all):,})')

Credits features ready: 45,432 rows
Keywords features ready: 45,432 rows

--- Creating 4 datasets ---
1. Metadata only:         5,352 rows
2. Metadata + Credits:    5,352 rows (lost 0)
3. Metadata + Keywords:   5,352 rows (lost 0)
4. All Combined:          5,352 rows (lost 0)


In [10]:
# 4.4  Top modern directors & actors (for datasets with credits)
# Computed from meta_credits (largest set with credits data)

# --- Top Directors (post-2010) ---
dir_year_pairs = []
for _, row in df_meta_credits.iterrows():
    year = row['release_year']
    crew = row['crew_parsed']
    if isinstance(crew, list):
        for m in crew:
            if isinstance(m, dict) and m.get('job') == 'Director' and m.get('name'):
                dir_year_pairs.append((m['name'], year))

dir_year_df = pd.DataFrame(dir_year_pairs, columns=['director', 'year'])
post2010_dirs = dir_year_df[dir_year_df['year'] >= 2010]['director'].value_counts()
top_directors = set(post2010_dirs.head(50).index)

def has_top_dir(crew):
    if not isinstance(crew, list): return 0
    for m in crew:
        if isinstance(m, dict) and m.get('job') == 'Director':
            if m.get('name', '') in top_directors: return 1
    return 0

# --- Top Actors (post-2010) ---
act_data = []
for _, row in df_meta_credits.iterrows():
    year = row['release_year']
    cast = row['cast_parsed']
    if isinstance(cast, list):
        for m in cast:
            if isinstance(m, dict) and m.get('name') and 'order' in m:
                act_data.append((m['name'], year, m['order']))

act_df = pd.DataFrame(act_data, columns=['actor', 'year', 'order'])
post2010_acts = act_df[act_df['year'] >= 2010]
top_actors = set(post2010_acts['actor'].value_counts().head(50).index)
top_leads = set(post2010_acts[post2010_acts['order'] == 0]['actor'].value_counts().head(50).index)

def count_top_actors(cast):
    if not isinstance(cast, list): return 0
    return sum(1 for m in cast if isinstance(m, dict) and m.get('name', '') in top_actors)

def check_top_lead(cast):
    if not isinstance(cast, list): return 0
    for m in cast:
        if isinstance(m, dict) and m.get('order') == 0:
            if m.get('name', '') in top_leads: return 1
    return 0

# Apply to both datasets with credits (meta_credits and all)
for df_label, df_ref in [('meta_credits', df_meta_credits), ('all', df_all)]:
    df_ref['has_top_director'] = df_ref['crew_parsed'].apply(has_top_dir)
    df_ref['has_top_actor'] = df_ref['cast_parsed'].apply(lambda x: 1 if count_top_actors(x) > 0 else 0)
    df_ref['top_actor_count'] = df_ref['cast_parsed'].apply(count_top_actors)
    df_ref['has_top_lead_actor'] = df_ref['cast_parsed'].apply(check_top_lead)
    n_td = df_ref['has_top_director'].sum()
    n_ta = df_ref['has_top_actor'].sum()
    print(f'{df_label}: has_top_director={n_td}, has_top_actor={n_ta}')

print(f'\nTop 10 modern directors: {post2010_dirs.head(10).index.tolist()}')
print(f'Top 5 modern actors sample: {list(top_actors)[:5]}')

meta_credits: has_top_director=475, has_top_actor=1184
all: has_top_director=475, has_top_actor=1184

Top 10 modern directors: ['Ridley Scott', 'Woody Allen', 'Lasse Hallström', 'Tim Burton', 'Shawn Levy', 'Nicholas Stoller', 'Jon M. Chu', 'Paul W.S. Anderson', 'Clint Eastwood', 'Michael Bay']
Top 5 modern actors sample: ['Bruce Willis', 'Bryan Cranston', 'Chris Hemsworth', 'Owen Wilson', 'Toby Kebbell']


In [11]:
# 4.5  Drop raw/unused columns from all 4 datasets
meta_drop = [
    'adult', 'video', 'status',
    'belongs_to_collection', 'homepage', 'tagline', 'poster_path',
    'imdb_id', 'original_title', 'overview',
    'original_language', 'release_date',
    'genres', 'production_companies', 'production_countries', 'spoken_languages',
]

# Dataset 1: Metadata Only
df_metadata = df_metadata.drop(columns=[c for c in meta_drop if c in df_metadata.columns])

# Dataset 2: Metadata + Credits
credits_drop = meta_drop + ['cast_parsed', 'crew_parsed']
df_meta_credits = df_meta_credits.drop(columns=[c for c in credits_drop if c in df_meta_credits.columns])

# Dataset 3: Metadata + Keywords
keywords_drop = meta_drop + ['keywords_parsed']
df_meta_keywords = df_meta_keywords.drop(columns=[c for c in keywords_drop if c in df_meta_keywords.columns])

# Dataset 4: All Combined
all_drop = meta_drop + ['cast_parsed', 'crew_parsed', 'keywords_parsed']
df_all = df_all.drop(columns=[c for c in all_drop if c in df_all.columns])

print('Columns per dataset:')
for name, df_ref in [('metadata', df_metadata), ('meta_credits', df_meta_credits),
                      ('meta_keywords', df_meta_keywords), ('all', df_all)]:
    print(f'\n  {name} ({df_ref.shape[0]:,} rows x {df_ref.shape[1]} cols):')
    for i, col in enumerate(df_ref.columns, 1):
        print(f'    {i:2d}. {col}')

Columns per dataset:

  metadata (5,352 rows x 17 cols):
     1. budget
     2. id
     3. popularity
     4. runtime
     5. title
     6. vote_average
     7. vote_count
     8. is_collection
     9. is_english
    10. release_year
    11. release_month
    12. roi
    13. num_genres
    14. num_production_companies
    15. num_production_countries
    16. num_spoken_languages
    17. primary_genre

  meta_credits (5,352 rows x 23 cols):
     1. budget
     2. id
     3. popularity
     4. runtime
     5. title
     6. vote_average
     7. vote_count
     8. is_collection
     9. is_english
    10. release_year
    11. release_month
    12. roi
    13. num_genres
    14. num_production_companies
    15. num_production_countries
    16. num_spoken_languages
    17. primary_genre
    18. num_cast
    19. num_crew
    20. has_top_director
    21. has_top_actor
    22. top_actor_count
    23. has_top_lead_actor

  meta_keywords (5,352 rows x 18 cols):
     1. budget
     2. id
     3. po

In [12]:
# 4.6  Dataset comparison overview
print('=' * 80)
print('DATASET COMPARISON')
print('=' * 80)

datasets_info = {
    'Metadata Only': df_metadata,
    'Meta + Credits': df_meta_credits,
    'Meta + Keywords': df_meta_keywords,
    'All Combined': df_all,
}

comparison_rows = []
for label, df_ref in datasets_info.items():
    numeric_df = df_ref.select_dtypes(include=[np.number])
    comparison_rows.append({
        'Dataset': label,
        'Rows': len(df_ref),
        'Columns': df_ref.shape[1],
        'Numeric Features': numeric_df.shape[1],
        'ROI Median': round(df_ref['roi'].median(), 1),
        'Profitable %': round((df_ref['roi'] > 0).mean() * 100, 1),
    })

comp_df = pd.DataFrame(comparison_rows)
print(comp_df.to_string(index=False))

# Missing values check
print('\n--- Missing Values ---')
for label, df_ref in datasets_info.items():
    n_missing = df_ref.isnull().sum().sum()
    print(f'{label}: {n_missing} total missing values')

DATASET COMPARISON
        Dataset  Rows  Columns  Numeric Features  ROI Median  Profitable %
  Metadata Only  5352       17                15       106.7          69.9
 Meta + Credits  5352       23                21       106.7          69.9
Meta + Keywords  5352       18                16       106.7          69.9
   All Combined  5352       24                22       106.7          69.9

--- Missing Values ---
Metadata Only: 0 total missing values
Meta + Credits: 0 total missing values
Meta + Keywords: 0 total missing values
All Combined: 0 total missing values


In [13]:
# 5. Save all 4 processed datasets
os.makedirs('../data/processed', exist_ok=True)

save_map = {
    'metadata':      (df_metadata,      '../data/processed/movies_processed_metadata.csv'),
    'meta_credits':  (df_meta_credits,  '../data/processed/movies_processed_meta_credits.csv'),
    'meta_keywords': (df_meta_keywords, '../data/processed/movies_processed_meta_keywords.csv'),
    'all':           (df_all,           '../data/processed/movies_processed.csv'),
}

print('Saving processed datasets:')
for key, (df_ref, path) in save_map.items():
    df_ref.to_csv(path, index=False)
    print(f'  ✓ {key:15s} → {path}  ({len(df_ref):,} rows x {df_ref.shape[1]} cols)')

Saving processed datasets:
  ✓ metadata        → ../data/processed/movies_processed_metadata.csv  (5,352 rows x 17 cols)
  ✓ meta_credits    → ../data/processed/movies_processed_meta_credits.csv  (5,352 rows x 23 cols)
  ✓ meta_keywords   → ../data/processed/movies_processed_meta_keywords.csv  (5,352 rows x 18 cols)
  ✓ all             → ../data/processed/movies_processed.csv  (5,352 rows x 24 cols)


---
## Summary

### 4 Processed Datasets Created

| Dataset | File | Sources |
|---|---|---|
| Metadata Only | `movies_processed_metadata.csv` | movies_metadata |
| Meta + Credits | `movies_processed_meta_credits.csv` | movies_metadata + credits |
| Meta + Keywords | `movies_processed_meta_keywords.csv` | movies_metadata + keywords |
| All Combined | `movies_processed.csv` | all three |

### Features Per Dataset

| Feature | Metadata | +Credits | +Keywords | All |
|---|:---:|:---:|:---:|:---:|
| `budget`, `popularity`, `runtime`, `vote_average`, `vote_count` | ✓ | ✓ | ✓ | ✓ |
| `is_collection`, `is_english` | ✓ | ✓ | ✓ | ✓ |
| `release_year`, `release_month` | ✓ | ✓ | ✓ | ✓ |
| `roi` | ✓ | ✓ | ✓ | ✓ |
| `num_genres`, `num_production_*`, `num_spoken_languages` | ✓ | ✓ | ✓ | ✓ |
| `primary_genre` | ✓ | ✓ | ✓ | ✓ |
| `num_cast`, `num_crew` | | ✓ | | ✓ |
| `has_top_director`, `has_top_actor`, `top_actor_count`, `has_top_lead_actor` | | ✓ | | ✓ |
| `num_keywords` | | | ✓ | ✓ |

### Next Steps
- **Notebook 03:** Prepare all 4 datasets for modeling — encode, split, scale, baselines